In [ ]:
quiet_library <- function(...) {
    suppressPackageStartupMessages(library(...))
}
quiet_library(Seurat)
quiet_library(ggplot2)
quiet_library(Matrix)
quiet_library(H5weaver)
quiet_library(dplyr)
quiet_library(viridis)
quiet_library(harmony)
quiet_library(Nebulosa)

In [ ]:
t_cells <- readRDS('../../09_Vax/clean_aging_t_cells.rds')

In [ ]:
t_cells

In [ ]:
table(t_cells$age)

In [ ]:
sort(unique(t_cells$pbmc_sample_id))

In [ ]:
t_cells <- SetIdent(t_cells, value = 'pbmc_sample_id')
t_cells <- RenameIdents(t_cells,
                        'PB00892-01'='M','PB00833-01'='M','PB00913-01'='M','PB00928-01'='M','PB00809-01'='F',
                        'PB00842-01'='M','PB00914-01'='M','PB00841-01'='F','PB00910-01'='F',
                        'PB00811-01'='F','PB00884-01'='M','PB00927-01'='F','PB02391-001'='F', 
                        'PB00193-01'='F','PB00026-05'='M','PB00010-02'='M','PB00003-01'='F',
                        'PB00031-05'='F','PB00507-01'='F','PB00002-01'='M','PB00384-01'='M',
                        'PB00006-01'='F','PB00023-05'='F','PB00041-01'='M',
                        'PB00022-01'='F','PB00363-01'='M','PB00386-01'='M','PB00366-01'='F',
                        'PB00030-03'='F','PB00004-01'='M','PB00552-01'='F','PB00356-01'='M',
                        'PB00530-01'='F','PB00510-01'='M','PB00352-04'='F','PB00015-01'='M',
                        'PB00388-01'='F','PB00539-01'='M','PB00569-01'='F','PB00342-01'='M','PB00566-01'='M',
                        'PB00537-01'='F','PB00561-01'='F','PB00562-01'='M','PB00568-01'='F',
                        'PB00582-01'='M','PB03223-001'='M','PB02392-001'='F')
t_cells$sex <- Idents(t_cells)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
DimPlot(t_cells, label = TRUE, repel = TRUE, group.by = 'SCT_snn_res.0.5')
DimPlot(t_cells, label = TRUE, repel = TRUE, group.by = 'sex')

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 20)
DimPlot(t_cells, split.by = 'batch_id', ncol = 4, group.by = 'SCT_snn_res.0.5')

In [ ]:
t_cells <- SetIdent(t_cells, value = 'SCT_snn_res.0.5')
t_cells <- FindSubCluster(object = t_cells, cluster = 12, graph.name = 'SCT_snn', resolution = 0.2)
Idents(t_cells) <- "sub.cluster"

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
DimPlot(t_cells, label = TRUE, repel = TRUE)
table(t_cells$sub.cluster)

In [ ]:
DefaultAssay(t_cells) <- 'RNA'
t_cells <- NormalizeData(t_cells)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 8)
VlnPlot(t_cells, features = c('KLRC2','ZNF683','CD3E','CD3G','TRGC2','TRDC',
                              'IKZF2','LEF1'), ncol = 2, pt.size = 0)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 12)
FeaturePlot(t_cells, features = c('KLRC2','ZNF683','IKZF2',
                                  'TRGC1','TRGC2','TRDC',
                                  'IL21R','CD8A','DUSP2'), ncol = 3, order = TRUE)

In [ ]:
mnp2_markers <- FindMarkers(t_cells, `ident.1` = '12_1', max.cells.per.ident = 500)

In [ ]:
head(subset(mnp2_markers, avg_log2FC > 0), n=20)

In [ ]:
ped_gdT_markers <- FindMarkers(t_cells, `ident.1` = '12_0', max.cells.per.ident = 500)

In [ ]:
head(subset(ped_gdT_markers, avg_log2FC > 0), n=20)

In [ ]:
cluster12_2 <- FindMarkers(t_cells, `ident.1` = '12_2', max.cells.per.ident = 500)

In [ ]:
head(subset(cluster12_2, avg_log2FC > 0), n=20)

In [ ]:
t_cells <- SetIdent(t_cells, value = 'age')
subset_50k <- subset(t_cells, downsample = 75000)

In [ ]:
p <- plot_density(subset_50k, c('IL21R','LEF1','KLRC2'), joint = TRUE)

## Fig 5I

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
p

In [ ]:
p <- plot_density(subset_50k, c('LEF1','DUSP2'), joint = TRUE)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
p + plot_layout(ncol = 2)

In [ ]:
p <- plot_density(subset_50k, c('LEF1','KLRC3','CD8A'), joint = TRUE)

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 12)
p + plot_layout(ncol = 2)

In [ ]:
# Open a pdf file
pdf("plots/mnp2_pipeline_markers_nebulosa_umap_May23.pdf", width = 12, height = 12) 
# 2. Create a plot
p
# Close the pdf file
dev.off() 

# CB MNP-2

In [ ]:
cb_ya_so <- readRDS('../../04_scRNA_Comparisons/Cord_blood/cb_ya_so.rds')

In [ ]:
table(cb_ya_so$orig.ident)
cb_ya_so <- SetIdent(cb_ya_so, value = 'orig.ident')
cb_only_so <- subset(cb_ya_so, idents = c('/home/jupyter/published_datasets/cord_blood_young_adult/GSM4750306_F016/',
                                          '/home/jupyter/published_datasets/cord_blood_young_adult/GSM4750307_F017/',
                                          '/home/jupyter/published_datasets/cord_blood_young_adult/GSM4750308_F024/'))

In [ ]:
table(cb_only_so$predicted.celltype.l1)

In [ ]:
cb_only_so <- SetIdent(cb_only_so, value = 'predicted.celltype.l1')
cb_only_t <- subset(cb_only_so, idents = c('CD4 T', 'CD8 T', 'other T'))

In [ ]:
DefaultAssay(cb_only_t) <- 'RNA'
cb_only_t <- NormalizeData(cb_only_t) %>% FindVariableFeatures() %>% ScaleData() %>% RunPCA()

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
ElbowPlot(cb_only_t, ndims = 50)

In [ ]:
cb_only_t <- RunHarmony(cb_only_t, group.by.vars = 'orig.ident')

In [ ]:
cb_only_t <- RunUMAP(cb_only_t, dims = 1:20, reduction = 'harmony') %>% FindNeighbors(dims = 1:20, reduction = 'harmony') %>% FindClusters(resolution = 0.5)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
DimPlot(cb_only_t, label = TRUE)
DimPlot(cb_only_t, group.by = 'predicted.celltype.l2', label = TRUE, repel = TRUE)
DimPlot(cb_only_t, group.by = 'orig.ident')

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
FeaturePlot(cb_only_t, features = c('LEF1','KLRC3','CD8A'), order = TRUE , ncol = 3)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
FeaturePlot(cb_only_t, features = c('TRDC','TRGC1','TRGC2'), order = TRUE , ncol = 3)

In [ ]:
cb_only_t <- SetIdent(cb_only_t, value = 'RNA_snn_res.0.5')
cb_only_t <- FindSubCluster(object = cb_only_t, cluster = 5, graph.name = 'RNA_snn', resolution = 0.2)
Idents(cb_only_t) <- "sub.cluster"

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
DimPlot(cb_only_t, label = TRUE)

In [ ]:
options(repr.plot.width = 20, repr.plot.height = 10)
VlnPlot(cb_only_t, features = c('LEF1','KLRC3','CD8A','TRDC','TRGC1','TRGC2'), ncol = 3)

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 10)
VlnPlot(cb_only_t, features = c('DDX3X','UTY','DDX3Y'), ncol = 3, group.by = 'orig.ident')

In [ ]:
# 'KLRC2','ZNF683','IKZF2',
# 'TRGC1','TRGC2','TRDC',
# 'IL21R','CD8A','DUSP2

In [ ]:
p[[4]]

In [ ]:
cb_only_t <- SetIdent(cb_only_t, value = 'sub.cluster')
mnp2_cb <- subset(cb_only_t, idents = '5_0')

In [ ]:
mnp2_counts_cb <- dplyr::count(mnp2_cb@meta.data, orig.ident)
# head(mnp2_counts_cb)
total_t_counts_cb <- dplyr::count(cb_only_t@meta.data, orig.ident)
# head(total_t_counts_cb)
joined_counts_cb <- left_join(mnp2_counts_cb, total_t_counts_cb, by = 'orig.ident')
# head(joined_counts_cb)
joined_counts_cb$perc <- joined_counts_cb$n.x / joined_counts_cb$n.y
head(joined_counts_cb)

In [ ]:
write.csv(joined_counts_cb, file = 'mnp2_cb_perc.csv')

In [ ]:
t_cells <- SetIdent(t_cells, value = 'sub.cluster')
mnp2_cluster <- subset(t_cells, idents = '12_1')

In [ ]:
mnp2_counts <- dplyr::count(mnp2_cluster@meta.data, pbmc_sample_id, age, sex)
head(mnp2_counts)

In [ ]:
total_t_counts <- dplyr::count(t_cells@meta.data, pbmc_sample_id)
head(total_t_counts)

In [ ]:
joined_counts <- left_join(mnp2_counts, total_t_counts, by = 'pbmc_sample_id')
head(joined_counts)

In [ ]:
joined_counts$perc <- joined_counts$n.x / joined_counts$n.y
head(joined_counts)

In [ ]:
write.csv(joined_counts, file = 'mnp2_scrna_seq_perc.csv')

In [ ]:
joined_counts <- read.csv(file = 'mnp2_scrna_seq_perc_full.csv')

In [ ]:
head(joined_counts)

In [ ]:
median(subset(joined_counts, age == 'Pediatric')$perc) *100
median(subset(joined_counts, age == 'Older Adult')$perc) *100

In [ ]:
mean(subset(joined_counts, age == 'Pediatric')$perc) * 100
mean(subset(joined_counts, age == 'Older Adult')$perc) * 100

## Fig 5J

In [ ]:
options(repr.plot.width = 14, repr.plot.height = 8)
ggplot(joined_counts, aes(x=Actual.Age, y=perc)) + 
    geom_point(size = 3, aes(color=age, shape=sex)) + 
    geom_smooth(formula=y ~ x^2,alpha=.15, se = TRUE, color='darkgrey') +
    scale_color_manual(values=c('#fa9fb5','#d95f02','#1b9e77','#9e9ac8')) + 
    xlab('Age') + ylab('% MNP-2 of Total T Cells') +
    theme_bw() + 
    theme(axis.text.x = element_text(size = 15),
          axis.text.y = element_text(size = 15),
          axis.title.y = element_text(size = 20),
          axis.title.x = element_text(size = 20),
          # legend.key.size = unit(2, 'cm'), #change legend key size
          # legend.key.height = unit(1, 'cm'), #change legend key height
          # legend.key.width = unit(1, 'cm'), #change legend key width
          legend.title = element_text(size=20), #change legend title font size
          legend.text = element_text(size=15))

In [ ]:
mnp2_counts <- dplyr::count(mnp2_cluster@meta.data, pbmc_sample_id, age, cmv)
head(mnp2_counts)

In [ ]:
total_t_counts <- dplyr::count(t_cells@meta.data, pbmc_sample_id)
head(total_t_counts)

In [ ]:
joined_counts <- left_join(mnp2_counts, total_t_counts, by = 'pbmc_sample_id')
head(joined_counts)

In [ ]:
joined_counts$perc <- joined_counts$n.x / joined_counts$n.y
head(joined_counts)

In [ ]:
write.csv(joined_counts, file = 'mnp2_scrna_seq_perc.csv')

In [ ]:
# Open a pdf file
pdf("plots/mnp2_perc_age_plot_wide.pdf", width = 14, height = 8) 
# 2. Create a plot
ggplot(joined_counts, aes(x=Actual.Age, y=perc)) + 
    geom_point(size = 3, aes(color=age, shape=sex)) + 
    geom_smooth(formula=y ~ x^2,alpha=.15, se = TRUE, color='darkgrey') +
    scale_color_manual(values=c('#fa9fb5','#d95f02','#1b9e77','#9e9ac8')) + 
    xlab('Age') + ylab('% MNP-2 of Total T Cells') +
    theme_bw() + 
    theme(axis.text.x = element_text(size = 15),
          axis.text.y = element_text(size = 15),
          axis.title.y = element_text(size = 20),
          axis.title.x = element_text(size = 20),
          # legend.key.size = unit(2, 'cm'), #change legend key size
          # legend.key.height = unit(1, 'cm'), #change legend key height
          # legend.key.width = unit(1, 'cm'), #change legend key width
          legend.title = element_text(size=20), #change legend title font size
          legend.text = element_text(size=15))
# Close the pdf file
dev.off() 

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
ggplot(joined_counts, aes(x=age, y=perc, color=sex)) + 
    geom_point() + 
    geom_smooth(alpha=.15, aes(fill=sex))

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
ggplot(joined_counts, aes(x=age, y=perc, color=sex)) + 
    geom_point() + 
    geom_smooth(method='lm',alpha=.15, aes(fill=sex))

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
ggplot(joined_counts, aes(x=age, y=perc, color=sex)) + 
    geom_point(size = 3) + 
    geom_smooth(formula=y ~ x^2,alpha=.15, aes(fill=sex), se = FALSE) + 
    scale_color_manual(values=c('#08519c','#9ecae1')) + 
    xlab('Age') + ylab('Fraction MNP-2 of Total T') +
    theme_bw() + 
    theme(axis.text.x = element_text(size = 15),
          axis.text.y = element_text(size = 15),
          axis.title.y = element_text(size = 20),
          axis.title.x = element_text(size = 20),
          # legend.key.size = unit(2, 'cm'), #change legend key size
          # legend.key.height = unit(1, 'cm'), #change legend key height
          # legend.key.width = unit(1, 'cm'), #change legend key width
          legend.title = element_text(size=20), #change legend title font size
          legend.text = element_text(size=15))

In [ ]:
# Open a pdf file
pdf("plots/mnp2_pipeline_age_sex_dotplot.pdf", width = 8, height = 8) 
# 2. Create a plot
ggplot(joined_counts, aes(x=age, y=perc, color=sex)) + 
    geom_point(size = 3) + 
    geom_smooth(formula=y ~ x^2,alpha=.15, aes(fill=sex), se = FALSE) + 
    scale_color_manual(values=c('#08519c','#9ecae1')) + 
    xlab('Age') + ylab('Fraction MNP-2 of Total T') +
    theme_bw() + 
    theme(axis.text.x = element_text(size = 15),
          axis.text.y = element_text(size = 15),
          axis.title.y = element_text(size = 20),
          axis.title.x = element_text(size = 20),
          # legend.key.size = unit(2, 'cm'), #change legend key size
          # legend.key.height = unit(1, 'cm'), #change legend key height
          # legend.key.width = unit(1, 'cm'), #change legend key width
          legend.title = element_text(size=20), #change legend title font size
          legend.text = element_text(size=15))
# Close the pdf file
dev.off() 

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
ggplot(joined_counts, aes(x=age, y=perc)) + 
    geom_boxplot() + 
    scale_y_log10() + 
    geom_dotplot(binaxis='y', stackdir='center',
                 position=position_dodge(), binwidth = 1/15)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
joined_counts %>% 
    mutate(age = factor(age, levels = c('Pediatric','Young Adult','Older Adult'))) %>%
    ggplot() + 
        geom_boxplot(aes(x=age, y=perc, fill=age)) +
        scale_fill_manual(values=c('#1b9e77','#de77ae','#d95f02')) +
        # scale_y_log10(breaks = ) + 
        scale_y_continuous(trans='log10',
                           breaks = c(0.0001,0.001,0.01,0.1),
                           labels = c(0.0001,0.001,0.01,0.1),
                           limits = c(0.0001,0.1)) + 
        geom_jitter(aes(x=age, y=perc, shape=sex, color=sex), border='black', size=5, width=0.2) +
        # geom_jitter(colour='black') +
        scale_color_manual(values=c('#3288bd','#d53e4f')) + 
        theme(axis.text.x = element_text(size = 20),
              axis.text.y = element_text(size = 15),
              axis.title.x = element_blank(),
              axis.title.y = element_blank())

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 8)
joined_counts %>% 
    mutate(age = factor(age, levels = c('Pediatric','Young Adult','Older Adult'))) %>%
    ggplot() + 
        geom_boxplot(aes(x=age, y=perc, fill=age)) +
        scale_fill_manual(values=c('#1b9e77','#de77ae','#d95f02')) +
        # scale_y_log10(breaks = ) + 
        scale_y_continuous(trans='log10',
                           breaks = c(0.0001,0.001,0.01,0.1),
                           labels = c(0.0001,0.001,0.01,0.1),
                           limits = c(0.0001,0.1)) + 
        geom_jitter(aes(x=age, y=perc, shape=sex), color='black', size=5, width=0.2) +
        # geom_jitter(colour='black') +
        # scale_color_manual(values=c('#3288bd','#d53e4f')) + 
        theme(axis.text.x = element_text(size = 20),
              axis.text.y = element_text(size = 15),
              axis.title.x = element_blank(),
              axis.title.y = element_blank(),
              panel.grid.major = element_blank(), 
              panel.grid.minor = element_blank(),
              panel.background = element_blank(),
              axis.line = element_line(colour = "black"))

In [ ]:
# Open a pdf file
pdf("plots/mnp2_pipeline_boxplot.pdf", width = 10, height = 8) 
# 2. Create a plot
joined_counts %>% 
    mutate(age = factor(age, levels = c('Pediatric','Young Adult','Older Adult'))) %>%
    ggplot() + 
        geom_boxplot(aes(x=age, y=perc, fill=age)) +
        scale_fill_manual(values=c('#1b9e77','#de77ae','#d95f02')) +
        # scale_y_log10(breaks = ) + 
        scale_y_continuous(trans='log10',
                           breaks = c(0.0001,0.001,0.01,0.1),
                           labels = c(0.0001,0.001,0.01,0.1),
                           limits = c(0.0001,0.1)) + 
        geom_jitter(aes(x=age, y=perc, shape=sex), color='black', size=5, width=0.2) +
        # geom_jitter(colour='black') +
        # scale_color_manual(values=c('#3288bd','#d53e4f')) + 
        theme(axis.text.x = element_text(size = 20),
              axis.text.y = element_text(size = 15),
              axis.title.x = element_blank(),
              axis.title.y = element_blank(),
              panel.grid.major = element_blank(), 
              panel.grid.minor = element_blank(),
              panel.background = element_blank(),
              axis.line = element_line(colour = "black"))
# Close the pdf file
dev.off() 

In [ ]:
t_cells <- SetIdent(t_cells, value = 'sub.cluster')
mnp2_cluster <- subset(t_cells, idents = c('12_1','12_2'))

In [ ]:
mnp2_counts <- dplyr::count(mnp2_cluster@meta.data, pbmc_sample_id, age, sex)
head(mnp2_counts)

In [ ]:
total_t_counts <- dplyr::count(t_cells@meta.data, pbmc_sample_id)
head(total_t_counts)

In [ ]:
joined_counts <- left_join(mnp2_counts, total_t_counts, by = 'pbmc_sample_id')
head(joined_counts)

In [ ]:
joined_counts$perc <- joined_counts$n.x / joined_counts$n.y
head(joined_counts)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
joined_counts %>% 
    mutate(age = factor(age, levels = c('Pediatric','Young Adult','Older Adult'))) %>%
    ggplot() + 
        geom_boxplot(aes(x=age, y=perc, fill=age)) +
        scale_fill_manual(values=c('#1b9e77','#de77ae','#d95f02')) +
        # scale_y_log10(breaks = ) + 
        scale_y_continuous(trans='log10',
                           breaks = c(0.0001,0.001,0.01,0.1),
                           labels = c(0.0001,0.001,0.01,0.1),
                           limits = c(0.0001,0.1)) + 
        geom_jitter(aes(x=age, y=perc, shape=sex), color='black', size=5, width=0.2) +
        # geom_jitter(colour='black') +
        # scale_color_manual(values=c('#3288bd','#d53e4f')) + 
        theme(axis.text.x = element_text(size = 20),
              axis.text.y = element_text(size = 15),
              axis.title.x = element_blank(),
              axis.title.y = element_blank())

In [ ]:
t_cells <- SetIdent(t_cells, value = 'sub.cluster')
mnp2_cluster <- subset(t_cells, idents = '12_1')
extra_cluster <- subset(t_cells, idents = '12_2')

In [ ]:
table(mnp2_cluster$age)
table(extra_cluster$age)